In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf

KeyboardInterrupt: 

In [ ]:
def list_classes_and_count(dataset_dir):
    classes = set()
    class_count = 0
    for root, dirs, files in os.walk(dataset_dir):
        for dir in dirs:
            classes.add(dir)
            class_count += 1
    return classes, class_count


# Directory paths for the two datasets
dataset_dir = "Plant Disease Recognition Dataset"

# List classes and count in each dataset
dataset_classes, dataset_classes = list_classes_and_count(dataset_dir + "/train")

# Display unique classes and the number of classes in each dataset
print("Classes in New Plant Diseases Dataset:")
print(dataset_classes)
print(f"Number of classes: {dataset_classes}")

Classes in New Plant Diseases Dataset:
3
Number of classes: 3


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Image data generators for both datasets with augmentation
datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2,
)

In [ ]:
BATCH_SIZE = 32
IMG_SIZE = (64, 64)


train_dataset = datagen.flow_from_directory(
    dataset_dir + "/train",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    subset="training",
)

valid_dataset = datagen.flow_from_directory(
    dataset_dir + "/valid",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    subset="validation",
)

Found 1059 images belonging to 3 classes.
Found 12 images belonging to 3 classes.


In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Concatenate
from tensorflow.keras.models import Model

# Load ResNet50 model without including the top fully-connected layers
IMG_SHAPE = IMG_SIZE + (3,)
base_model = ResNet50(input_shape=IMG_SHAPE, weights="imagenet", include_top=False)

base_model.trainable = True

# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# Fine-tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

Number of layers in the base model:  175


In [ ]:
# Let's take a look at the base model architecture
base_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_18 (InputLayer)       [(None, 64, 64, 3)]          0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 70, 70, 3)            0         ['input_18[0][0]']            
                                                                                                  
 conv1_conv (Conv2D)         (None, 32, 32, 64)           9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 32, 32, 64)           256       ['conv1_conv[0][0]']          
 on)                                                                                       

In [ ]:
# Add custom top layers for classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation="relu")(x)
# x = tf.keras.layers.Dropout(0.2)(x)
predictions = Dense(3, activation="softmax")(x)

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
base_learning_rate = 0.001

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

In [ ]:
model.summary()

Model: "model_16"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_18 (InputLayer)       [(None, 64, 64, 3)]          0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 70, 70, 3)            0         ['input_18[0][0]']            
                                                                                                  
 conv1_conv (Conv2D)         (None, 32, 32, 64)           9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 32, 32, 64)           256       ['conv1_conv[0][0]']          
 on)                                                                                       

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor="val_accuracy", patience=5, restore_best_weights=True, verbose=1
)

In [ ]:
# Train the model
history = model.fit(
    train_dataset,
    epochs=100,
    validation_data=valid_dataset,
    callbacks=[early_stopping],
)

Epoch 1/100
34/34 [==============================] - 91s 2s/step - loss: 1.4238 - accuracy: 0.3428 - val_loss: 25.0423 - val_accuracy: 0.3333
Epoch 2/100
34/34 [==============================] - 72s 2s/step - loss: 1.1295 - accuracy: 0.3343 - val_loss: 6.5188 - val_accuracy: 0.3333
Epoch 3/100
34/34 [==============================] - 72s 2s/step - loss: 1.1129 - accuracy: 0.3456 - val_loss: 1.1889 - val_accuracy: 0.3333
Epoch 4/100
34/34 [==============================] - 72s 2s/step - loss: 1.1043 - accuracy: 0.3447 - val_loss: 1.1249 - val_accuracy: 0.3333
Epoch 5/100
34/34 [==============================] - 72s 2s/step - loss: 1.0952 - accuracy: 0.3607 - val_loss: 1.0994 - val_accuracy: 0.3333
Epoch 6/100
34/34 [==============================] - 72s 2s/step - loss: 1.0900 - accuracy: 0.3834 - val_loss: 1.0988 - val_accuracy: 0.3333
Epoch 6: early stopping
